In [ ]:
### TCGA

In [1]:
library(TCGAbiolinks)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
projects <- getGDCprojects()
projects_tcga <- projects$id[grep("TCGA",projects$id)]
dir_project_tcga <- tolower(gsub("-","_",projects_tcga))

In [3]:
funct_read_patients_sample_file <- function(x) {
    file <- paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/02_selection_samples/02_1_ext_information_patients_tcga/", x, "/patients_geneexp_clind_drug_samptype_",x,".rds",sep="")
    patients_sample_type <- readRDS(file)
    patients_sample_type$project <- rep(x, length(patients_sample_type$patient))
    return(patients_sample_type)
}

In [4]:
patients_sample <- do.call(rbind,lapply(dir_project_tcga[-6], FUN=function(x) funct_read_patients_sample_file(x)))
colnames(patients_sample)                                        

[1] "bcr_patient_barcode"              "days_to_birth"                   
 [3] "gender"                           "vital_status"                    
 [5] "days_to_death"                    "days_to_last_followup"           
 [7] "has_new_tumor_events_information" "drug_name"                       
 [9] "age"                              "patient"                         
[11] "sample_type"                      "project"

In [5]:
projects_filt_samples_tcga <- as.data.frame(patients_sample %>% select(gender,has_new_tumor_events_information,sample_type,patient,project)  %>% filter(sample_type == "Primary Solid Tumor") %>%
group_by(gender, has_new_tumor_events_information, project) %>% summarise(n=n()) %>% filter(n >2) %>% group_by(project) %>% summarise(n=n()) %>% filter(n == 4) %>% select(project))$project

`summarise()` has grouped output by 'gender', 'has_new_tumor_events_information'. You can override using the `.groups` argument.


In [ ]:
### GTEX

In [31]:
donors_sex_age_tissue_gtex <- readRDS("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/02_selection_samples/02_2_ext_information_patients_gtex/donors_sex_age_tissue.rds")

In [36]:
donors_sex_age_tissue_gtex %>% group_by(sex,tissue) %>% summarise(n=n()) %>% filter(n >2) %>% group_by(tissue) %>% summarise(n=n()) %>% filter(n == 2)

`summarise()` has grouped output by 'sex'. You can override using the `.groups` argument.


tissue,n
<chr>,<int>
Adipose Tissue,2
Adrenal Gland,2
Blood,2
Blood Vessel,2
Brain,2
Breast,2
Colon,2
Esophagus,2
Heart,2


In [ ]:
### selection:

# tcga_brca - breast
# tcga_coad - colon
# tcga_esca - esophagus
# tcga_lgg - brain
# tcga_luad - lung
# tcga_lusc - lung
# tcga_paad - pancreas
# tcga_skcm - skin
# tcga_stad - stomach